In [52]:
import requests
from dotenv import load_dotenv 
import os
import json

In [53]:
load_dotenv()

True

In [54]:
def make_post_request(payload:dict):
    resp = requests.post(
        os.getenv('HARPERDB_URL'),
        json=payload,
        auth=(os.getenv('HARPERDB_USERNAME'), os.getenv('HARPERDB_PASSWORD'))
    )

    resp.raise_for_status()
    return resp.json()

In [65]:
def create_table(tbl_schema:str, tbl_name:str):
    payload = {
        "operation": "create_table",
        "schema": tbl_schema,
        "table": tbl_name,
        "primary_key": "timestamp",
        "attributes": [
            {
                "name": "timestamp",
                "type": "Date",
                "indexed": True
            },
            {
                "name": "event",
                "type": "string",
                "indexed": True
            },
            {
                "name": "percent",
                "type": "Int",
                "indexed": False
            }
        ]
    }

    return make_post_request(payload)

In [55]:
def get_records(sql_query:str):
    # HarperDB's REST API expects a JSON payload with the operation type and query
    payload = {
        "operation": "sql",
        "sql": sql_query
    }    
    return make_post_request(payload)

In [67]:
def describe_table(tbl_schema:str, tbl_name:str):
    payload = {
        'operation': 'describe_table',
        'schema': tbl_schema,
        'table': tbl_name
    }
    return make_post_request(payload)    

In [56]:
def drop_table(tbl_schema:str, tbl_name:str):
    payload = {
        'operation': 'drop_table',
        'schema': tbl_schema,
        'table': tbl_name
    }
    return make_post_request(payload)

In [58]:
# simple query
query = f'select * from {os.getenv("HARPERDB_SCHEMA")}.{os.getenv("HARPERDB_TABLE")};'

In [62]:
print(json.dumps(get_records(query)[6], indent=2))

HTTPError: 404 Client Error: Not Found for url: http://localhost:9925/

In [68]:
#describe table
describe_table(tbl_schema=os.getenv("HARPERDB_SCHEMA"), tbl_name=os.getenv('HARPERDB_TABLE'))

{'schema': 'analytics',
 'name': 'events',
 'hash_attribute': 'timestamp',
 'audit': True,
 'schema_defined': True,
 'attributes': [{'attribute': 'timestamp',
   'type': 'Date',
   'indexed': True,
   'is_primary_key': True},
  {'attribute': 'event', 'type': 'string', 'indexed': True},
  {'attribute': 'percent', 'type': 'Int', 'indexed': False}],
 'db_size': 917504,
 'sources': [],
 'record_count': 0,
 'table_size': 0,
 'db_audit_size': 16384}

In [64]:
# drop the dummy table created so far
resp = drop_table(tbl_schema=os.getenv('HARPERDB_SCHEMA'),tbl_name=os.getenv('HARPERDB_TABLE'))
print(resp)

HTTPError: 404 Client Error: Not Found for url: http://localhost:9925/

In [66]:
create_table(tbl_schema=os.getenv('HARPERDB_SCHEMA'),tbl_name=os.getenv('HARPERDB_TABLE'))

{'message': "table 'analytics.events' successfully created."}